#### As discussed in the paper, we stacked the datasets into sets of 10. 

In [ ]:
from functools import reduce
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sklearn.preprocessing as pp
from scipy.ndimage import gaussian_filter, gaussian_filter1d
from scipy.signal import triang
import torch
from torch import nn
import time
from tqdm import tqdm
from scipy.signal import decimate
from scipy import interpolate
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from numpy.matlib import repmat
from tqdm import tqdm
from itertools import combinations

In [ ]:
CO2_Maps = np.load('data/co2maps/co2_maps.npy')
Seismic_Gathers = np.load('data/seismic_gathers/seismic_gathers.npy')

In [ ]:
# combination

#split first
X_main_npa,X_test_npa,y_main_npa,y_test_npa = train_test_split(Seismic_Gathers,CO2_Maps,test_size=0.05,random_state=155)
X_train_npa,X_valid_npa,y_train_npa,y_valid_npa = train_test_split(X_main_npa,y_main_npa,test_size=0.05,random_state=155)

idx_array = np.arange(20)
comb_list = np.array(list(combinations(idx_array,10)))
np.random.seed(155)
X_train_np = []; y_train_np = []
X_valid_np = []; y_valid_np = [] 
X_test_np = []; y_test_np = []

#do training data
for i in tqdm(range(len(X_train_npa))):
    stemp = []; ctemp = []
    Ncomb = 300
    idx_comb = np.arange(len(comb_list))
    Ncomb_list = np.random.choice(idx_comb,size=Ncomb,replace=False)
    for j in range(Ncomb):
        selected_comb_idx = Ncomb_list[j]
        selected_comb = comb_list[selected_comb_idx]
        sp1 = X_train_npa[i,selected_comb]
        cp1 = y_train_npa[i,selected_comb] 
        stemp.append(sp1)
        ctemp.append(cp1)
    stemp = np.array(stemp)
    ctemp = np.array(ctemp)
    X_train_np.append(stemp)
    y_train_np.append(ctemp)
X_train_np = np.array(X_train_np)
y_train_np = np.array(y_train_np)

#do validation data
for i in tqdm(range(len(X_valid_npa))):
    stemp = []; ctemp = []
    Ncomb = 500
    idx_comb = np.arange(len(comb_list))
    Ncomb_list = np.random.choice(idx_comb,size=Ncomb,replace=False)
    for j in range(Ncomb):
        selected_comb_idx = Ncomb_list[j]
        selected_comb = comb_list[selected_comb_idx]
        sp1 = X_valid_npa[i,selected_comb]
        cp1 = y_valid_npa[i,selected_comb] 
        stemp.append(sp1)
        ctemp.append(cp1)
    stemp = np.array(stemp)
    ctemp = np.array(ctemp)
    X_valid_np.append(stemp)
    y_valid_np.append(ctemp)
X_valid_np = np.array(X_valid_np)
y_valid_np = np.array(y_valid_np)


#do testing data
for i in tqdm(range(len(X_test_npa))):
    stemp = []; ctemp = []
    Ncomb = 500
    idx_comb = np.arange(len(comb_list))
    Ncomb_list = np.random.choice(idx_comb,size=Ncomb,replace=False)
    for j in range(Ncomb):
        selected_comb_idx = Ncomb_list[j]
        selected_comb = comb_list[selected_comb_idx]
        sp1 = X_test_npa[i,selected_comb]
        cp1 = y_test_npa[i,selected_comb] 
        stemp.append(sp1)
        ctemp.append(cp1)
    stemp = np.array(stemp)
    ctemp = np.array(ctemp)
    X_test_np.append(stemp)
    y_test_np.append(ctemp)
X_test_np = np.array(X_test_np)
y_test_np = np.array(y_test_np)


In [ ]:
X_train_np = X_train_np.reshape(X_train_np.shape[0]*X_train_np.shape[1],X_train_np.shape[2],X_train_np.shape[3],X_train_np.shape[4])
y_train_np = y_train_np.reshape(y_train_np.shape[0]*y_train_np.shape[1],y_train_np.shape[2],y_train_np.shape[3],y_train_np.shape[4])
X_valid_np = X_valid_np.reshape(X_valid_np.shape[0]*X_valid_np.shape[1],X_valid_np.shape[2],X_valid_np.shape[3],X_valid_np.shape[4])
y_valid_np = y_valid_np.reshape(y_valid_np.shape[0]*y_valid_np.shape[1],y_valid_np.shape[2],y_valid_np.shape[3],y_valid_np.shape[4])
X_test_np = X_test_np.reshape(X_test_np.shape[0]*X_test_np.shape[1],X_test_np.shape[2],X_test_np.shape[3],X_test_np.shape[4])
y_test_np = y_test_np.reshape(y_test_np.shape[0]*y_test_np.shape[1],y_test_np.shape[2],y_test_np.shape[3],y_test_np.shape[4])

In [ ]:
np.save('data/crop_DLdatasets/exF_comb_X_train.npy',X_train_np)
np.save('data/crop_DLdatasets/exF_comb_X_valid.npy',X_valid_np)
np.save('data/crop_DLdatasets/exF_comb_y_train.npy',y_train_np)
np.save('data/crop_DLdatasets/exF_comb_y_valid.npy',y_valid_np)
np.save('data/crop_DLdatasets/exF_comb_X_test.npy',X_test_np)
np.save('data/crop_DLdatasets/exF_comb_y_test.npy',y_test_np)
